# SQL Server 2019 Polybase - Data Virtualization
This notebook has a series of instructions to setup external data sources using Polybase in SQL Server 2019

## SQL Server Polybase with Oracle
The following will go through how to setup an external data source, external table, and query to Oracle. First change context to the WideWorldImporters database and remove an existing external tables from a previous run.

### Step 1: Add a master key

Add a master key to encrypt the database scoped credential

In [36]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'S0me!nfo'
GO

: Msg 15578, Level 16, State 1, Line 1
There is already a master key in the database. Please drop it before performing this statement.

Total execution time: 00:00:00.0028134

### Step 2: Create database scoped credentials

In [37]:
/*  specify credentials to external data source
*  IDENTITY: user name for external source.  
*  SECRET: password for external source.
*/
DROP DATABASE SCOPED CREDENTIAL OracleCredentials
GO
CREATE DATABASE SCOPED CREDENTIAL OracleCredentials   
WITH IDENTITY = 'gl', Secret = 'glpwd'
GO

Commands completed successfully.

Total execution time: 00:00:00.0217615

Commands completed successfully.

Total execution time: 00:00:00.0310924

### 

### Step 3: Create the external data source

In [38]:
/*  LOCATION: Location string should be of format '<vendor>://<server>[:<port>]'.
*  PUSHDOWN: specify whether computation should be pushed down to the source. ON by default.
*  CREDENTIAL: the database scoped credential, created above.
*/  
CREATE EXTERNAL DATA SOURCE OracleServer
WITH ( 
LOCATION = 'oracle://bworacle:49161',
PUSHDOWN = ON,
CREDENTIAL = OracleCredentials
)
GO


Commands completed successfully.

Total execution time: 00:00:00.1757543

### Step 4: Create a new schema

In [39]:
DROP SCHEMA oracle
go
CREATE SCHEMA oracle
GO

Commands completed successfully.

Total execution time: 00:00:00.0846266

Commands completed successfully.

Total execution time: 00:00:00.0157469

### Step 5: Create the exteranl table

In [40]:
/*  LOCATION: oracle table/view in 'database_name.schema_name.object_name' format
*  DATA_SOURCE: the external data source, created above.
*/
CREATE EXTERNAL TABLE oracle.accountsreceivable
(
arid int,
ardate date,
ardesc nvarchar(100) COLLATE Latin1_General_100_CI_AS,
arref int,
aramt decimal(10,2)
)
 WITH (
 LOCATION='[XE].[GL].[ACCOUNTSRECEIVABLE]',
 DATA_SOURCE=OracleServer
)

Commands completed successfully.

Total execution time: 00:00:00.6258433

### Step 6: Create statistics on key columns

In [41]:
CREATE STATISTICS arrefstats ON oracle.accountsreceivable ([arref]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:04.4838751

### Step 7: Do a filter query for predicate pushdown

In [42]:
-- Try a simple filter
SELECT * FROM oracle.accountsreceivable
WHERE arref = 336252
GO

(1 row affected)

Total execution time: 00:00:06.2927365

arid,ardate,ardesc,arref,aramt
1,2016-05-31,Invoice 70510,336252,5549.90


### Step 8: Join with a local table

In [43]:
-- Join with a local table
--
SELECT ct.*, oa.arid, oa.ardesc
FROM oracle.accountsreceivable oa
JOIN [Sales].[CustomerTransactions] ct
ON oa.arref = ct.CustomerTransactionID
GO

(1 row affected)

Total execution time: 00:00:04.8095964

CustomerTransactionID,CustomerID,TransactionTypeID,InvoiceID,PaymentMethodID,TransactionDate,AmountExcludingTax,TaxAmount,TransactionAmount,OutstandingBalance,FinalizationDate,IsFinalized,LastEditedBy,LastEditedWhen,arid,ardesc
336252,401,1,70510,NULL,2016-05-31,4826.00,723.90,5549.90,5549.90,NULL,0,3,2016-05-31 12:00:00.0000000,1,Invoice 70510
